In [78]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from collections import OrderedDict

#remove warnings
import warnings
warnings.filterwarnings('ignore')

In [79]:
#get player teams into dataframe from player_teams.csv
player_teams = pd.read_csv('../data/players_teams.csv')

#get player teams into dataframe from player_teams.csv
coaches = pd.read_csv('../data/coaches.csv')

#teams match up results (only post is available)
series_post = pd.read_csv('../data/series_post.csv')

#team stats in playoff
teams_post = pd.read_csv('../data/teams_post.csv')

#player csv
players = pd.read_csv('../data/players.csv')

#awards csv
awards_players = pd.read_csv('../data/awards_players.csv')

#teams csv
teams = pd.read_csv('../data/teams.csv')

#remove all columns starting with "lgID" from all dataframes
for df in [player_teams, coaches, series_post, teams_post, players, awards_players, teams]:
    for col in df.columns:
        if col.startswith('lgID'):
            df.drop(col, axis=1, inplace=True)

#drop divID column from teams
teams.drop('divID', axis=1, inplace=True)

In [80]:
def compute_percentage(numerator, denominator):
    return round(numerator.divide(denominator).where(denominator != 0, 0.0),2)

In [81]:
# Assuming you have a column 'year' to sort by
player_teams = player_teams.sort_values(by=['playerID', 'year'])
player_teams['career_year'] = player_teams.groupby('playerID').cumcount() + 1

# Regular Season Percentages
player_teams['ft%'] = compute_percentage(player_teams['ftMade'], player_teams['ftAttempted'])
player_teams['fg%'] = compute_percentage(player_teams['fgMade'], player_teams['fgAttempted'])
player_teams['three%'] = compute_percentage(player_teams['threeMade'], player_teams['threeAttempted'])
player_teams['gs%'] = compute_percentage(player_teams['GS'], player_teams['GP'])

# Playoffs Percentages
player_teams['Postft%'] = compute_percentage(player_teams['PostftMade'], player_teams['PostftAttempted'])
player_teams['Postfg%'] = compute_percentage(player_teams['PostfgMade'], player_teams['PostfgAttempted'])
player_teams['Postthree%'] = compute_percentage(player_teams['PostthreeMade'], player_teams['PostthreeAttempted'])
player_teams['Postgs%'] = compute_percentage(player_teams['PostGS'], player_teams['PostGP'])

#effective field goal percentage
player_teams['efg%'] = compute_percentage(player_teams['fgMade'] + 0.5 * player_teams['threeMade'], player_teams['fgAttempted']) * 2

#true shooting percentage
player_teams['ts%'] = compute_percentage(player_teams['points'], 2 * (player_teams['fgAttempted'] + 0.44 * player_teams['ftAttempted'])) * 2

#per game stats
#if pos contains G, then multiply by assists by 1.5, if contains C, then multiply by rebounds by 1.5, if contains F, then points by 1.5 (it may contain more than one letter ex: G-F )
player_teams['ppg'] = round(player_teams['points']/player_teams['GP'],2)
player_teams['rpg'] = round(player_teams['rebounds']/player_teams['GP'],2)
player_teams['apg'] = round(player_teams['assists']/player_teams['GP'],2)
player_teams['spg'] = round(player_teams['steals']/player_teams['GP'],2)
player_teams['bpg'] = round(player_teams['blocks']/player_teams['GP'],2)

#efficiency
player_teams['eff'] = player_teams['ppg'] + player_teams['rpg'] + player_teams['apg'] + player_teams['spg'] + player_teams['bpg'] - (player_teams['fgAttempted'] - player_teams['fgMade']) - (player_teams['ftAttempted'] - player_teams['ftMade']) - player_teams['turnovers']

#per 36 minutes stats
player_teams['pp36'] = compute_percentage(player_teams['points'], player_teams['minutes'])*36

#defensive prowess: Defensive Prowess PCA: Use 'steals', 'blocks', and 'dRebounds' to create a 'Defensive Impact' principal component. Combine 'PF' (personal fouls) and 'turnovers' into a 'Defensive Discipline' component to represent careful play.
player_teams['defensive_prowess'] = compute_percentage(player_teams['steals'] + player_teams['blocks'] + player_teams['dRebounds'], player_teams['GP'])*10
player_teams['defensive_discipline'] = compute_percentage(player_teams['PF'] + player_teams['turnovers'], player_teams['GP'])*2

#minutes per game
player_teams['mpg'] = compute_percentage(player_teams['minutes'], player_teams['GP'])

#percetange of game started
player_teams['gs%'] = compute_percentage(player_teams['GS'], player_teams['GP'])
    

#add pos column from players to player_teams, bioID is the same as playerID
player_teams = player_teams.merge(players[['bioID', 'pos', 'college']], left_on='playerID', right_on='bioID', how='left')
player_teams.drop('bioID', axis=1, inplace=True)

player_teams['pos'] = player_teams['pos'].replace(
    ['G', 'F', 'C', 'C-F', 'F-C', 'G-F', 'F-G'],
    [1, 2, 3, 4, 4, 5, 5]
)

#change college to an id, if none, then -1
player_teams['college'] = player_teams['college'].fillna(-1)
player_teams['college'] = player_teams['college'].astype('category').cat.codes

#turn this oRebounds,dRebounds,dq,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostDQ into per game stats
player_teams['oRebounds'] = round(player_teams['oRebounds']/player_teams['GP'],2)
player_teams['dRebounds'] = round(player_teams['dRebounds']/player_teams['GP'],2)
player_teams['dq'] = round(player_teams['dq']/player_teams['GP'],2)
player_teams['PostMinutes'] = round(player_teams['PostMinutes']/player_teams['PostGP'],2)
player_teams['PostPoints'] = round(player_teams['PostPoints']/player_teams['PostGP'],2)
player_teams['PostoRebounds'] = round(player_teams['PostoRebounds']/player_teams['PostGP'],2)
player_teams['PostdRebounds'] = round(player_teams['PostdRebounds']/player_teams['PostGP'],2)
player_teams['PostRebounds'] = round(player_teams['PostRebounds']/player_teams['PostGP'],2)
player_teams['PostAssists'] = round(player_teams['PostAssists']/player_teams['PostGP'],2)
player_teams['PostSteals'] = round(player_teams['PostSteals']/player_teams['PostGP'],2)
player_teams['PostBlocks'] = round(player_teams['PostBlocks']/player_teams['PostGP'],2)
player_teams['PostTurnovers'] = round(player_teams['PostTurnovers']/player_teams['PostGP'],2)
player_teams['PostPF'] = round(player_teams['PostPF']/player_teams['PostGP'],2)
player_teams['PostDQ'] = round(player_teams['PostDQ']/player_teams['PostGP'],2)

#remove the made and attempted columns
player_teams.drop([ 'threeMade', 'threeAttempted', 'PostftMade', 'PostftAttempted', 'PostfgMade', 'PostfgAttempted', 'PostthreeMade', 'PostthreeAttempted', 'PostGS', 'PostGP'], axis=1, inplace=True)

#remove the columns that are not needed: points, rebounds, assists, steals, blocks, turnovers, PF, fgMade, fgAttempted, ftMade, ftAttempted, GS, GP, minutes
player_teams.drop([ 'points', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgMade', 'fgAttempted', 'ftMade', 'ftAttempted', 'GS', 'minutes'], axis=1, inplace=True)

#fill nan with 0
player_teams.fillna(0, inplace=True)

player_teams.head()
player_teams.columns

Index(['playerID', 'year', 'stint', 'tmID', 'GP', 'oRebounds', 'dRebounds',
       'dq', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds',
       'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks',
       'PostTurnovers', 'PostPF', 'PostDQ', 'career_year', 'ft%', 'fg%',
       'three%', 'gs%', 'Postft%', 'Postfg%', 'Postthree%', 'Postgs%', 'efg%',
       'ts%', 'ppg', 'rpg', 'apg', 'spg', 'bpg', 'eff', 'pp36',
       'defensive_prowess', 'defensive_discipline', 'mpg', 'pos', 'college'],
      dtype='object')

In [82]:
coaches['total_games'] = coaches['won']+ coaches['lost']
coaches['W%'] = compute_percentage(coaches['won'],coaches['total_games'])

coaches['total_p_games'] = coaches['post_wins']+ coaches['post_losses']
coaches['postW%'] = compute_percentage(coaches['post_wins'],coaches['total_p_games'])

#remove won and lost columns, and post_wins and post_losses
coaches.drop(['won','lost','post_wins','post_losses'], axis=1, inplace=True)

coaches.head()

,coachID,year,tmID,stint,total_games,W%,total_p_games,postW%
0,adamsmi01w,5,WAS,0,34,0.50,3,0.33
1,adubari99w,1,NYL,0,32,0.62,7,0.57
2,adubari99w,2,NYL,0,32,0.66,6,0.50
3,adubari99w,3,NYL,0,32,0.56,8,0.50
4,adubari99w,4,NYL,0,34,0.47,0,0.00


In [83]:
def pre_process_data(df):
    mapping = {'Y': 1, 'N': 0}
    df['playoff'] = df['playoff'].map(mapping)
    df.fillna(0, inplace=True)
    return df

In [84]:
teams = pre_process_data(teams)

In [85]:
#just get the columns tmID, year, playoff, confID, firstRound, semis, finals
teams_playoffs = teams[['tmID', 'year', 'playoff', 'confID', 'firstRound', 'semis', 'finals']]

#merge teams_playoffs with player_teams, based on tmID and year, and add the playoff columns to player_teams
player_teams = player_teams.merge(teams_playoffs, left_on=['tmID', 'year'], right_on=['tmID', 'year'], how='left')

#add champions column to player_teams
player_teams['champions'] = player_teams.apply(
    lambda row: 'W' if row['finals'] == 'W' else '',
    axis=1
)
# Create a new column for playoff_progression
player_teams['playoff_progression'] = player_teams.apply(
    lambda row: '1' if row['firstRound'] == 'L' else
                '2' if row['semis'] == 'L' else
                '3' if row['finals'] == 'L' else
                '4' if row['champions'] == 'W' else '0',
    axis=1
)
# Drop unnecessary columns
player_teams.drop(['firstRound', 'semis', 'finals', 'champions'], axis=1, inplace=True)

player_teams.head()

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,eff,pp36,defensive_prowess,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression
0,abrossv01w,2,0,MIN,26,1.65,5.04,0.08,0.0,0.00,...,-276.11,14.76,70.0,11.92,32.54,2,20,0,WE,0
1,abrossv01w,3,0,MIN,27,1.67,3.74,0.00,0.0,0.00,...,-327.81,14.04,56.7,12.22,29.81,2,20,0,WE,0
2,abrossv01w,4,0,MIN,30,1.47,3.23,0.00,23.0,7.67,...,-272.13,14.40,50.7,11.26,26.40,2,20,1,WE,1
3,abrossv01w,5,0,MIN,22,0.77,2.59,0.00,33.5,10.00,...,-137.50,11.52,40.5,7.72,21.00,2,20,1,WE,1
4,abrossv01w,6,0,MIN,31,0.94,2.52,0.00,0.0,0.00,...,-250.06,14.04,42.6,10.70,25.06,2,20,0,WE,0


In [86]:
#add height and weight columns to player_teams from players
player_teams = player_teams.merge(players[['bioID', 'height', 'weight']], left_on='playerID', right_on='bioID', how='left')

#remove bioID column
player_teams.drop('bioID', axis=1, inplace=True)

player_teams.head()

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,defensive_prowess,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight
0,abrossv01w,2,0,MIN,26,1.65,5.04,0.08,0.0,0.00,...,70.0,11.92,32.54,2,20,0,WE,0,74.0,169
1,abrossv01w,3,0,MIN,27,1.67,3.74,0.00,0.0,0.00,...,56.7,12.22,29.81,2,20,0,WE,0,74.0,169
2,abrossv01w,4,0,MIN,30,1.47,3.23,0.00,23.0,7.67,...,50.7,11.26,26.40,2,20,1,WE,1,74.0,169
3,abrossv01w,5,0,MIN,22,0.77,2.59,0.00,33.5,10.00,...,40.5,7.72,21.00,2,20,1,WE,1,74.0,169
4,abrossv01w,6,0,MIN,31,0.94,2.52,0.00,0.0,0.00,...,42.6,10.70,25.06,2,20,0,WE,0,74.0,169


In [87]:
#from awards_players, add column award_count to player_teams which is the number of awards the team won (sum of all awards)
#player_teams = player_teams.merge(awards_players[['playerID', 'award']], left_on='playerID', right_on='playerID', how='left')

player_teams['award_count'] = 0

#associate each player with each award and year (ex: player A won 2 awards in 2010, thus 2010 has 2, but 2011 is 0 (unless he wins again))
for index, row in awards_players.iterrows():
    player_teams.loc[(player_teams['playerID'] == row['playerID']) & (player_teams['year'] == row['year']), 'award_count'] += 1

player_teams.head()

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight,award_count
0,abrossv01w,2,0,MIN,26,1.65,5.04,0.08,0.0,0.00,...,11.92,32.54,2,20,0,WE,0,74.0,169,0
1,abrossv01w,3,0,MIN,27,1.67,3.74,0.00,0.0,0.00,...,12.22,29.81,2,20,0,WE,0,74.0,169,0
2,abrossv01w,4,0,MIN,30,1.47,3.23,0.00,23.0,7.67,...,11.26,26.40,2,20,1,WE,1,74.0,169,0
3,abrossv01w,5,0,MIN,22,0.77,2.59,0.00,33.5,10.00,...,7.72,21.00,2,20,1,WE,1,74.0,169,0
4,abrossv01w,6,0,MIN,31,0.94,2.52,0.00,0.0,0.00,...,10.70,25.06,2,20,0,WE,0,74.0,169,0


In [88]:
#store the player_teams dataframe into a csv file
player_teams.to_csv('new_data/clean-data.csv', index=False)